In [1]:

import sys

command_str = '{sys.executable} -m pip freeze'.format(sys=sys)
print(command_str)
!{command_str}

C:\Users\dev\Anaconda3\envs\pt\python.exe -m pip freeze
absl-py==0.11.0
argon2-cffi @ file:///C:/ci/argon2-cffi_1596828549974/work
async-generator==1.10
attrs @ file:///tmp/build/80754af9/attrs_1600298409949/work
backcall==0.2.0
bleach @ file:///tmp/build/80754af9/bleach_1600439572647/work
boto3==1.16.10
botocore==1.19.10
cachetools==4.1.1
certifi==2020.6.20
cffi @ file:///C:/ci/cffi_1600699250966/work
chardet==3.0.4
click==7.1.2
colorama @ file:///tmp/build/80754af9/colorama_1603211150991/work
Cython @ file:///C:/ci/cython_1594834055134/work
decorator==4.4.2
defusedxml==0.6.0
entrypoints==0.3
fairseq @ git+https://github.com/pytorch/fairseq@e6422528dae0b899848469efe2dc404c1e639ce9
fasttext==0.8.3
filelock==3.0.12
future==0.18.2
google-auth==1.23.0
google-auth-oauthlib==0.4.2
grpcio==1.33.2
hypothesis==3.88.3
idna==2.10
importlib-metadata @ file:///tmp/build/80754af9/importlib-metadata_1602276842396/work
ipykernel @ file:///C:/ci/ipykernel_1596208728219/work/dist/ipykernel-5.3.4-py3-no

In [2]:

%run ../../load_magic/environment.py
import transformers

get_module_version(transformers)

__version__: 3.4.0


In [1]:

from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample
from torch.utils.data import DataLoader

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
   InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

In [2]:

import re

scanner_regex = re.compile(r'(</?[^><]+>|\w+:?)')
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]

In [44]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cv = CountVectorizer(lowercase=False, tokenizer=regex_tokenizer, ngram_range=(1, 3))
def tokenize_texts(df, match_series):
    sents_list = df.navigable_parent.tolist()
    
    # Bag-of-words
    bow_matrix = cv.fit_transform(sents_list)

    # Get everything into a list of lists of ints
    pro_bow_lil = bow_matrix[match_series].toarray().tolist()
    anti_bow_lil = bow_matrix[~match_series].toarray().tolist()
    
    # Get one float for the label
    pro_labels_list = df[match_series].is_header.unique().tolist()
    assert len(pro_labels_list) == 1, f'Pro-sentences should all have the same label, instead we got {pro_labels_list}'
    pro_label = float(pro_labels_list[0])
    anti_labels_list = df[~match_series].is_header.unique().tolist()
    assert len(anti_labels_list) == 1, f'Anti-sentences should all have the same label, instead we got {anti_labels_list}'
    anti_label = float(anti_labels_list[0])
    
    return (pro_bow_lil, anti_bow_lil, pro_label, anti_label)

In [45]:

import pandas as pd
%run ../../load_magic/storage.py
s = Storage()

# Re-transform the bag-of-words and tf-idf from the new manual scores
child_str_dict = s.load_object('child_str_dict')
rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in child_str_dict.items()]
child_str_df = pd.DataFrame(rows_list)
match_series = (child_str_df.is_header == True)
header_texts_tokenized, nonheader_texts_tokenized, header_label, nonheader_label = tokenize_texts(child_str_df, match_series)

In [46]:

type(header_texts_tokenized[0][0])

int

In [47]:

isinstance(type(header_texts_tokenized[0][0]), int)

False

In [48]:

from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

#Define your train examples
train_examples = [InputExample(texts_tokenized=header_texts_tokenized, label=header_label),
                  InputExample(texts_tokenized=nonheader_texts_tokenized, label=nonheader_label)]

#Define your train dataset, the dataloader and the train loss
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3219456 bytes. Buy new RAM!
